# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [48]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [49]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = str(cells[2]).replace("<br/>","/").replace("</td>","").replace("<td>","") # Athlete's name, splitted by "/" if it's a team
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            sex = cells[5].text.strip()  # Sex
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event,
                'sex' : sex
                })

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)

url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"
# Try with one country - FRA (France)
html = GetHTML(url)

Getdataframe(html)

Response OK, continue.


,medal,country,athlete_name,games,sport,event,sex
0,1,France,Paul MASSON,Greece,Cycling Track,sprint individual,Men's
1,1,France,Paul MASSON,Greece,Cycling Track,1/3km time trial,Men's
2,1,France,Paul MASSON,Greece,Cycling Track,10km,Men's
3,1,France,Léon FLAMENG,Greece,Cycling Track,100km,Men's
4,1,France,Eugène-Henri GRAVELOTTE,Greece,Fencing,foil individual,Men's
...,...,...,...,...,...,...,...
884,2,France,SIMON Julia/FILLON MAILLET Quentin/CHEVALIER-B...,China,Biathlon,Relay mix,Mixed
885,2,France,LEDEUX Tess,China,Freestyle Skiing,Big air,Women's
886,2,France,TRESPEUCH Chloe,China,Snowboard,Snowboard Cross,Women's
887,3,France,FAIVRE Mathieu,China,Alpine Skiing,giant slalom,Men's


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [50]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [51]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [52]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event,sex
562,2,France,Клинье,United States,Cycling Track,individual pursuit,Women's
826,3,France,RAYNAUD Alexis,Brazil,Shooting,50m rifle 3 positions,Men's
624,3,France,,Australia,Rowing,lightweight double sculls (2x),Men's
578,3,France,Коваль/Барата,United States,Rowing,double sculls (2x),Men's
432,2,France,Daniel SENET,Canada,Weightlifting,60 - 67.5kg (lightweight),Men's


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [53]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [54]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [55]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 and len(data[0]) == 3:
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [56]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
114,NGR,Nigeria,Africa,1952,2022
74,JPN,Japan,Asia,1912,2022
137,ESP,Spain,Europe,1900,2022
28,CAN,Canada,America,1900,2022
127,ROU,Romania,Europe,1900,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [60]:
# Run scraping on all countries 
code_countries = df_countries["Code"]
name_countries = df_countries["Country"]
i = 0

for country, country_name in zip(code_countries, name_countries) :
    url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
    html = GetHTML(url)
    df_temp = Getdataframe(html)
    if i == 0 :
        df_olympic = df_temp
    else : 
        df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

    #Delete these codes after tries
    if i > 5 :
        break

    i += 1
    
    print("for : {}".format(country_name))

Response OK, continue.
for : Afghanistan
Response OK, continue.
for : Albania
Response OK, continue.
for : Algeria
Response OK, continue.
for : Andorra
Response OK, continue.
for : Angola
Response OK, continue.
for : Argentina
Response OK, continue.


In [ ]:
df_olympic.sample(15)

,medal,country,athlete_name,games,sport,event,sex
93,2,Argentina,SANCHEZ MOCCIA Rocio/SUCCI Maria Belen/TRINCHI...,Japan,Hockey,hockey,Women's
543,3,Australia,Lisa OLDENHOF/Lyndsie FOGARTY/Chantal MEEK/Han...,China,Canoe Sprint,K-4 500m (kayak four),Women's
597,3,Australia,Lydia IERODIACONOU,Russian Federation,Freestyle Skiing,aerials,Women's
1079,1,Bahamas,Паулин Дэвис,Australia,Athletics,200m,Women's
242,3,Australia,Ian O'BRIEN/Peter REYNOLDS/David DICKSON/Kevin...,Japan,Swimming,4x100m medley relay,Men's
553,3,Australia,Cate CAMPBELL/Libby TRICKETT/Alice MILLS/Shayn...,China,Swimming,4x100m freestyle relay,Women's
999,3,Austria,Lukas KLAPFER,Korea Republic,Nordic Combined,individual,Men's
209,1,Australia,David THEILE,Italy,Swimming,100m backstroke,Men's
1281,3,Belgium,René MORTIAUX/Victor VERSCHUEREN/Henri P. WILL...,France,Bobsleigh,four-man,Men's
1091,1,Bahrain,Maryam Yusuf JAMAL,Great Britain,Athletics,1500m,Women's


In [62]:
len(df_olympic.event.unique())

71

To do :
- Get simple sport list
- Get simple event list (weight, lenght, heigh, etc. split ?)
- Look for empty name (sometimes filled with country name)